# Music Library Manager

**Difficulty**: ⭐⭐
**Estimated Time**: 30 minutes
**Prerequisites**: Basic Python knowledge, file system operations

## Learning Objectives
By the end of this notebook, you will be able to:
1. Scan and analyze your music library structure
2. Search for songs to avoid duplicate downloads
3. Detect duplicate files using various methods
4. Organize new downloads into appropriate folders automatically
5. Perform batch rename operations on music files
6. Move files based on naming patterns and folder similarity

## Overview
This notebook provides tools to manage your music library efficiently. It helps you:
- **Check before downloading**: Search if a song already exists
- **Organize automatically**: Move new downloads to the right folders
- **Batch operations**: Rename or move multiple files at once
- **Find duplicates**: Identify potential duplicate songs

## 1. Setup and Configuration

In [ ]:
# Import required libraries
import os
import shutil
from pathlib import Path
from collections import defaultdict, Counter
from typing import List, Dict, Tuple
import re
from difflib import SequenceMatcher
import hashlib
from datetime import datetime

# For better display
import pandas as pd
from IPython.display import display, HTML

# Configuration
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Configure your music library path
# Change this to your actual music folder path
MUSIC_LIBRARY_PATH = Path('../../music')

# Supported audio file extensions
AUDIO_EXTENSIONS = {'.mp3', '.flac', '.wav', '.m4a', '.aac', '.ogg', '.wma', '.opus'}

# Verify the path exists
if not MUSIC_LIBRARY_PATH.exists():
    print(f"⚠️ Warning: Music library path does not exist: {MUSIC_LIBRARY_PATH.absolute()}")
    print(f"Creating directory...")
    MUSIC_LIBRARY_PATH.mkdir(parents=True, exist_ok=True)
else:
    print(f"✓ Music library path: {MUSIC_LIBRARY_PATH.absolute()}")

## 2. Core Functions: Scanning and Searching

In [ ]:
def scan_music_library(library_path: Path) -> List[Dict]:
    """
    Scan the music library and return a list of all audio files with metadata.
    
    Returns:
        List of dictionaries containing file information
    """
    music_files = []
    
    # Walk through all directories and subdirectories
    for root, dirs, files in os.walk(library_path):
        for file in files:
            file_path = Path(root) / file
            
            # Check if it's an audio file
            if file_path.suffix.lower() in AUDIO_EXTENSIONS:
                # Get file information
                stat_info = file_path.stat()
                
                music_files.append({
                    'filename': file,
                    'path': str(file_path),
                    'folder': str(file_path.parent.relative_to(library_path)),
                    'size_mb': stat_info.st_size / (1024 * 1024),
                    'modified': datetime.fromtimestamp(stat_info.st_mtime),
                    'extension': file_path.suffix.lower()
                })
    
    return music_files


def search_songs(query: str, music_files: List[Dict], 
                 case_sensitive: bool = False) -> pd.DataFrame:
    """
    Search for songs by name in the music library.
    
    Args:
        query: Search term
        music_files: List of music file dictionaries
        case_sensitive: Whether search should be case-sensitive
    
    Returns:
        DataFrame with matching songs
    """
    if not case_sensitive:
        query = query.lower()
    
    results = []
    for file in music_files:
        filename = file['filename'] if case_sensitive else file['filename'].lower()
        
        if query in filename:
            results.append(file)
    
    if results:
        df = pd.DataFrame(results)
        # Format size for better readability
        df['size_mb'] = df['size_mb'].round(2)
        return df
    else:
        return pd.DataFrame(columns=['filename', 'path', 'folder', 'size_mb', 'modified'])


def get_library_stats(music_files: List[Dict]) -> Dict:
    """
    Calculate statistics about the music library.
    """
    if not music_files:
        return {
            'total_files': 0,
            'total_size_gb': 0,
            'by_format': {},
            'by_folder': {}
        }
    
    total_size = sum(f['size_mb'] for f in music_files)
    formats = Counter(f['extension'] for f in music_files)
    folders = Counter(f['folder'] for f in music_files)
    
    return {
        'total_files': len(music_files),
        'total_size_gb': round(total_size / 1024, 2),
        'by_format': dict(formats),
        'by_folder': dict(folders)
    }

print("✓ Core functions loaded")

## 3. Duplicate Detection

In [ ]:
def calculate_file_hash(file_path: Path, chunk_size: int = 8192) -> str:
    """
    Calculate MD5 hash of a file for exact duplicate detection.
    
    Args:
        file_path: Path to the file
        chunk_size: Size of chunks to read (for memory efficiency)
    
    Returns:
        MD5 hash as hexadecimal string
    """
    md5_hash = hashlib.md5()
    
    with open(file_path, 'rb') as f:
        # Read file in chunks to handle large files
        for chunk in iter(lambda: f.read(chunk_size), b''):
            md5_hash.update(chunk)
    
    return md5_hash.hexdigest()


def find_exact_duplicates(music_files: List[Dict]) -> Dict[str, List[Dict]]:
    """
    Find exact duplicate files by comparing file hashes.
    This is the most reliable method but slower for large libraries.
    
    Returns:
        Dictionary mapping hash to list of duplicate files
    """
    print("Calculating file hashes (this may take a while for large libraries)...")
    
    hash_to_files = defaultdict(list)
    
    for i, file in enumerate(music_files, 1):
        if i % 100 == 0:
            print(f"  Processed {i}/{len(music_files)} files...")
        
        file_hash = calculate_file_hash(Path(file['path']))
        hash_to_files[file_hash].append(file)
    
    # Keep only hashes with duplicates
    duplicates = {h: files for h, files in hash_to_files.items() if len(files) > 1}
    
    print(f"✓ Found {len(duplicates)} sets of exact duplicates")
    return duplicates


def find_similar_names(music_files: List[Dict], threshold: float = 0.8) -> List[Tuple]:
    """
    Find files with similar names (potential duplicates).
    Uses fuzzy string matching to detect similar filenames.
    
    Args:
        threshold: Similarity threshold (0-1, higher = more similar)
    
    Returns:
        List of tuples (file1, file2, similarity_score)
    """
    similar_pairs = []
    
    # Compare each file with every other file
    for i, file1 in enumerate(music_files):
        for file2 in music_files[i+1:]:
            # Use SequenceMatcher to calculate similarity
            similarity = SequenceMatcher(
                None, 
                file1['filename'].lower(), 
                file2['filename'].lower()
            ).ratio()
            
            if similarity >= threshold:
                similar_pairs.append((file1, file2, round(similarity, 2)))
    
    return similar_pairs


def find_same_size_duplicates(music_files: List[Dict]) -> Dict[float, List[Dict]]:
    """
    Find files with the same size (quick potential duplicate check).
    
    Returns:
        Dictionary mapping file size to list of files
    """
    size_to_files = defaultdict(list)
    
    for file in music_files:
        size_to_files[file['size_mb']].append(file)
    
    # Keep only sizes with duplicates
    duplicates = {s: files for s, files in size_to_files.items() if len(files) > 1}
    
    return duplicates

print("✓ Duplicate detection functions loaded")

## 4. Organization and Auto-Sorting

In [ ]:
def suggest_folder(filename: str, existing_folders: List[str], 
                   threshold: float = 0.6) -> Tuple[str, float]:
    """
    Suggest which folder a file should go into based on filename similarity.
    
    Args:
        filename: Name of the file to organize
        existing_folders: List of existing folder names
        threshold: Minimum similarity score to suggest a folder
    
    Returns:
        Tuple of (suggested_folder, confidence_score)
    """
    best_match = None
    best_score = 0
    
    # Clean filename for comparison (remove extension, special chars)
    clean_name = Path(filename).stem.lower()
    clean_name = re.sub(r'[^a-z0-9\s]', ' ', clean_name)
    
    for folder in existing_folders:
        if folder == '.':
            continue
            
        clean_folder = folder.lower()
        
        # Calculate similarity
        similarity = SequenceMatcher(None, clean_name, clean_folder).ratio()
        
        # Check if filename contains folder name or vice versa
        if clean_folder in clean_name or clean_name in clean_folder:
            similarity = max(similarity, 0.8)
        
        if similarity > best_score:
            best_score = similarity
            best_match = folder
    
    if best_score >= threshold:
        return best_match, round(best_score, 2)
    else:
        return None, 0


def organize_new_downloads(source_path: Path, library_path: Path, 
                          existing_folders: List[str], 
                          dry_run: bool = True) -> List[Dict]:
    """
    Organize new music downloads into appropriate folders.
    
    Args:
        source_path: Path containing new downloads
        library_path: Main music library path
        existing_folders: List of existing folder names
        dry_run: If True, only show what would be done without moving files
    
    Returns:
        List of actions taken/suggested
    """
    actions = []
    
    # Find all audio files in source
    for file_path in source_path.rglob('*'):
        if file_path.is_file() and file_path.suffix.lower() in AUDIO_EXTENSIONS:
            # Suggest folder
            suggested_folder, confidence = suggest_folder(
                file_path.name, 
                existing_folders
            )
            
            if suggested_folder:
                destination = library_path / suggested_folder / file_path.name
                
                action = {
                    'file': file_path.name,
                    'from': str(file_path),
                    'to': str(destination),
                    'confidence': confidence,
                    'status': 'Would move' if dry_run else 'Moved'
                }
                
                if not dry_run:
                    # Create folder if it doesn't exist
                    destination.parent.mkdir(parents=True, exist_ok=True)
                    # Move file
                    shutil.move(str(file_path), str(destination))
                
                actions.append(action)
            else:
                actions.append({
                    'file': file_path.name,
                    'from': str(file_path),
                    'to': 'No suitable folder found',
                    'confidence': 0,
                    'status': 'Needs manual review'
                })
    
    return actions

print("✓ Organization functions loaded")

## 5. Batch Rename Operations

In [ ]:
def batch_rename(folder_path: Path, pattern: str, replacement: str, 
                 dry_run: bool = True) -> List[Dict]:
    """
    Batch rename files matching a pattern.
    
    Args:
        folder_path: Path to folder containing files to rename
        pattern: Regex pattern to match in filenames
        replacement: Replacement string
        dry_run: If True, show what would be renamed without actually renaming
    
    Returns:
        List of rename actions
    """
    actions = []
    
    for file_path in folder_path.rglob('*'):
        if file_path.is_file() and file_path.suffix.lower() in AUDIO_EXTENSIONS:
            old_name = file_path.name
            
            # Apply pattern replacement
            new_name = re.sub(pattern, replacement, old_name)
            
            if new_name != old_name:
                new_path = file_path.parent / new_name
                
                action = {
                    'folder': str(file_path.parent.relative_to(folder_path)),
                    'old_name': old_name,
                    'new_name': new_name,
                    'status': 'Would rename' if dry_run else 'Renamed'
                }
                
                if not dry_run:
                    # Check if target already exists
                    if new_path.exists():
                        action['status'] = 'Skipped (target exists)'
                    else:
                        file_path.rename(new_path)
                
                actions.append(action)
    
    return actions


def batch_add_prefix(folder_path: Path, prefix: str, dry_run: bool = True) -> List[Dict]:
    """
    Add a prefix to all audio files in a folder.
    
    Args:
        folder_path: Path to folder
        prefix: Prefix to add
        dry_run: If True, show what would be done without actually renaming
    """
    actions = []
    
    for file_path in folder_path.iterdir():
        if file_path.is_file() and file_path.suffix.lower() in AUDIO_EXTENSIONS:
            old_name = file_path.name
            new_name = f"{prefix}{old_name}"
            new_path = file_path.parent / new_name
            
            action = {
                'old_name': old_name,
                'new_name': new_name,
                'status': 'Would rename' if dry_run else 'Renamed'
            }
            
            if not dry_run:
                if new_path.exists():
                    action['status'] = 'Skipped (target exists)'
                else:
                    file_path.rename(new_path)
            
            actions.append(action)
    
    return actions


def batch_clean_names(folder_path: Path, dry_run: bool = True) -> List[Dict]:
    """
    Clean filenames by removing common unwanted patterns.
    Removes things like: [Official Video], (Audio), extra spaces, etc.
    """
    # Common patterns to remove
    patterns_to_remove = [
        r'\[Official Video\]',
        r'\[Official Audio\]',
        r'\(Official Video\)',
        r'\(Official Audio\)',
        r'\[HD\]',
        r'\[HQ\]',
        r'\(HD\)',
        r'\(HQ\)',
        r'\s+\-\s+',  # Multiple spaces around dash
        r'\s{2,}'     # Multiple consecutive spaces
    ]
    
    actions = []
    
    for file_path in folder_path.rglob('*'):
        if file_path.is_file() and file_path.suffix.lower() in AUDIO_EXTENSIONS:
            old_name = file_path.name
            new_name = old_name
            
            # Remove each pattern
            for pattern in patterns_to_remove:
                new_name = re.sub(pattern, '', new_name, flags=re.IGNORECASE)
            
            # Clean up any resulting issues
            new_name = new_name.strip()
            new_name = re.sub(r'\s{2,}', ' ', new_name)  # Remove double spaces
            
            if new_name != old_name:
                new_path = file_path.parent / new_name
                
                action = {
                    'folder': str(file_path.parent.relative_to(folder_path)),
                    'old_name': old_name,
                    'new_name': new_name,
                    'status': 'Would rename' if dry_run else 'Renamed'
                }
                
                if not dry_run:
                    if new_path.exists():
                        action['status'] = 'Skipped (target exists)'
                    else:
                        file_path.rename(new_path)
                
                actions.append(action)
    
    return actions

print("✓ Batch rename functions loaded")

## 6. Batch Move Operations

In [ ]:
def batch_move_by_pattern(source_folder: Path, target_folder: Path, 
                          pattern: str, dry_run: bool = True) -> List[Dict]:
    """
    Move files matching a pattern to a target folder.
    
    Args:
        source_folder: Folder to search in
        target_folder: Destination folder
        pattern: Regex pattern to match filenames
        dry_run: If True, show what would be moved without actually moving
    """
    actions = []
    
    for file_path in source_folder.rglob('*'):
        if file_path.is_file() and file_path.suffix.lower() in AUDIO_EXTENSIONS:
            if re.search(pattern, file_path.name, re.IGNORECASE):
                destination = target_folder / file_path.name
                
                action = {
                    'file': file_path.name,
                    'from': str(file_path.parent),
                    'to': str(target_folder),
                    'status': 'Would move' if dry_run else 'Moved'
                }
                
                if not dry_run:
                    target_folder.mkdir(parents=True, exist_ok=True)
                    if destination.exists():
                        action['status'] = 'Skipped (target exists)'
                    else:
                        shutil.move(str(file_path), str(destination))
                
                actions.append(action)
    
    return actions


def batch_move_similar_to_folders(library_path: Path, threshold: float = 0.7, 
                                   dry_run: bool = True) -> List[Dict]:
    """
    Move files from root to folders with similar names.
    For example: "Beatles - Hey Jude.mp3" → "Beatles/" folder
    
    Args:
        library_path: Music library path
        threshold: Similarity threshold for matching
        dry_run: If True, show what would be done
    """
    actions = []
    
    # Get all existing folders (potential targets)
    existing_folders = [f for f in library_path.iterdir() 
                       if f.is_dir() and not f.name.startswith('.')]
    folder_names = [f.name for f in existing_folders]
    
    # Find files in root directory
    for file_path in library_path.iterdir():
        if file_path.is_file() and file_path.suffix.lower() in AUDIO_EXTENSIONS:
            # Try to find matching folder
            suggested_folder, confidence = suggest_folder(
                file_path.name, 
                folder_names,
                threshold
            )
            
            if suggested_folder:
                destination = library_path / suggested_folder / file_path.name
                
                action = {
                    'file': file_path.name,
                    'to_folder': suggested_folder,
                    'confidence': confidence,
                    'status': 'Would move' if dry_run else 'Moved'
                }
                
                if not dry_run:
                    if destination.exists():
                        action['status'] = 'Skipped (target exists)'
                    else:
                        shutil.move(str(file_path), str(destination))
                
                actions.append(action)
    
    return actions

print("✓ Batch move functions loaded")

---
## 7. Usage Examples

Below are practical examples of using the music library manager.

### 7.1 Scan Your Music Library

In [ ]:
# Scan the entire music library
print("Scanning music library...")
all_music_files = scan_music_library(MUSIC_LIBRARY_PATH)

print(f"\n✓ Found {len(all_music_files)} audio files")

# Show a sample if files exist
if all_music_files:
    sample_df = pd.DataFrame(all_music_files[:10])
    display(sample_df[['filename', 'folder', 'size_mb', 'extension']])
else:
    print("\nNo music files found. Add some music to your library to get started!")

### 7.2 View Library Statistics

In [ ]:
# Get library statistics
stats = get_library_stats(all_music_files)

print("📊 Music Library Statistics\n" + "="*50)
print(f"Total Files: {stats['total_files']}")
print(f"Total Size: {stats['total_size_gb']} GB")

if stats['by_format']:
    print(f"\nFiles by Format:")
    for fmt, count in sorted(stats['by_format'].items(), key=lambda x: x[1], reverse=True):
        print(f"  {fmt}: {count} files")

if stats['by_folder']:
    print(f"\nFiles by Folder (top 10):")
    sorted_folders = sorted(stats['by_folder'].items(), key=lambda x: x[1], reverse=True)[:10]
    for folder, count in sorted_folders:
        print(f"  {folder}: {count} files")

### 7.3 Search Before Downloading

**Use Case**: Before downloading a new song, check if you already have it.

In [ ]:
# Example: Search for a song
search_query = "your_song_name_here"  # Change this to search for a specific song

print(f"Searching for: '{search_query}'...\n")
results = search_songs(search_query, all_music_files)

if len(results) > 0:
    print(f"✓ Found {len(results)} matching song(s):\n")
    display(results[['filename', 'folder', 'size_mb']])
    print("\n⚠️ You already have this song! No need to download.")
else:
    print("✓ Song not found in your library. Safe to download!")

### 7.4 Find Potential Duplicates

**Use Case**: Clean up your library by finding duplicate files.

In [ ]:
# Quick check: Find files with the same size
if all_music_files:
    print("Checking for files with identical sizes...\n")
    same_size = find_same_size_duplicates(all_music_files)
    
    if same_size:
        print(f"Found {len(same_size)} groups of files with identical sizes:\n")
        for size, files in list(same_size.items())[:5]:  # Show first 5 groups
            print(f"Size: {size:.2f} MB ({len(files)} files)")
            for f in files:
                print(f"  - {f['filename']} ({f['folder']})")
            print()
    else:
        print("✓ No files with identical sizes found.")
else:
    print("No music files to check.")

In [ ]:
# Find files with similar names (fuzzy matching)
if all_music_files and len(all_music_files) > 1:
    print("Checking for files with similar names...\n")
    
    # Limit to first 100 files to avoid long processing time
    sample = all_music_files[:100] if len(all_music_files) > 100 else all_music_files
    similar = find_similar_names(sample, threshold=0.85)
    
    if similar:
        print(f"Found {len(similar)} pairs of similar files:\n")
        for file1, file2, similarity in similar[:10]:  # Show first 10
            print(f"Similarity: {similarity*100}%")
            print(f"  1. {file1['filename']} ({file1['folder']})")
            print(f"  2. {file2['filename']} ({file2['folder']})")
            print()
    else:
        print("✓ No similar filenames found.")
else:
    print("Not enough files to compare.")

### 7.5 Batch Rename Files

**Use Case**: Clean up filenames by removing unwanted text.

In [ ]:
# Example: Clean filenames in a specific folder
# First, let's see what would be cleaned (dry run)

folder_to_clean = MUSIC_LIBRARY_PATH  # Change to specific folder if needed

print("Preview: Cleaning filenames (dry run)...\n")
cleanup_actions = batch_clean_names(folder_to_clean, dry_run=True)

if cleanup_actions:
    print(f"Would clean {len(cleanup_actions)} files:\n")
    df = pd.DataFrame(cleanup_actions[:10])  # Show first 10
    display(df)
    
    print("\n💡 To actually rename these files, run:")
    print("   cleanup_actions = batch_clean_names(folder_to_clean, dry_run=False)")
else:
    print("✓ No files need cleaning.")

In [ ]:
# Example: Custom pattern replacement
# Replace a specific pattern in filenames

# Uncomment and customize these lines:
# pattern = r"old_text"  # Regex pattern to find
# replacement = "new_text"  # Replacement text
# actions = batch_rename(MUSIC_LIBRARY_PATH, pattern, replacement, dry_run=True)
# display(pd.DataFrame(actions))

print("💡 Tip: Use regex patterns for powerful renaming:")
print("   - r'\\s+' matches multiple spaces")
print("   - r'\\(.*?\\)' matches text in parentheses")
print("   - r'^prefix' matches text at start of filename")

### 7.6 Auto-Organize Files to Folders

**Use Case**: Automatically move files from root to appropriate subfolders based on name similarity.

In [ ]:
# Move files from root to folders with similar names
print("Analyzing files for auto-organization...\n")

organize_actions = batch_move_similar_to_folders(
    MUSIC_LIBRARY_PATH, 
    threshold=0.7,  # Adjust threshold as needed (0.5-0.9)
    dry_run=True
)

if organize_actions:
    print(f"Would organize {len(organize_actions)} files:\n")
    df = pd.DataFrame(organize_actions)
    display(df)
    
    print("\n💡 To actually move these files, run:")
    print("   organize_actions = batch_move_similar_to_folders(")
    print("       MUSIC_LIBRARY_PATH, threshold=0.7, dry_run=False)")
else:
    print("✓ No files need organizing or all files are already in folders.")

### 7.7 Move Files by Pattern

**Use Case**: Move all files matching a pattern to a specific folder.

In [ ]:
# Example: Move all files matching a pattern to a folder

# Uncomment and customize:
# source = MUSIC_LIBRARY_PATH
# target = MUSIC_LIBRARY_PATH / "Artist Name"  # Create folder if needed
# pattern = r"artist.*name"  # Regex pattern to match
#
# actions = batch_move_by_pattern(source, target, pattern, dry_run=True)
# if actions:
#     display(pd.DataFrame(actions))
# else:
#     print("No files match the pattern.")

print("💡 Example patterns:")
print("   - r'beatles' - matches any file with 'beatles' in name")
print("   - r'^beatles' - matches files starting with 'beatles'")
print("   - r'beatles.*live' - matches 'beatles' followed by 'live'")

## 8. Complete Workflow Example

Here's a complete workflow for managing new downloads:

In [ ]:
def manage_new_downloads_workflow(download_folder: Path, library_path: Path):
    """
    Complete workflow for processing new music downloads.
    
    Steps:
    1. Scan downloads folder
    2. Check for duplicates
    3. Clean filenames
    4. Organize into appropriate folders
    """
    print("🎵 Music Download Management Workflow")
    print("="*60 + "\n")
    
    # Step 1: Scan downloads
    print("Step 1: Scanning downloads folder...")
    downloads = scan_music_library(download_folder)
    print(f"  Found {len(downloads)} files\n")
    
    if not downloads:
        print("No new downloads to process.")
        return
    
    # Step 2: Check for duplicates
    print("Step 2: Checking for duplicates in existing library...")
    existing = scan_music_library(library_path)
    
    for new_file in downloads:
        matches = search_songs(Path(new_file['filename']).stem, existing)
        if len(matches) > 0:
            print(f"  ⚠️ Duplicate found: {new_file['filename']}")
    print()
    
    # Step 3: Clean filenames
    print("Step 3: Cleaning filenames...")
    clean_actions = batch_clean_names(download_folder, dry_run=False)
    print(f"  Cleaned {len(clean_actions)} filenames\n")
    
    # Step 4: Organize into folders
    print("Step 4: Organizing into folders...")
    existing_folders = [f.name for f in library_path.iterdir() if f.is_dir()]
    organize_actions = organize_new_downloads(
        download_folder, 
        library_path,
        existing_folders,
        dry_run=False
    )
    
    if organize_actions:
        print(f"  Organized {len(organize_actions)} files")
        display(pd.DataFrame(organize_actions))
    else:
        print("  No files to organize\n")
    
    print("\n✓ Workflow complete!")

# Example usage (uncomment to use):
# downloads_path = Path("/path/to/downloads")
# manage_new_downloads_workflow(downloads_path, MUSIC_LIBRARY_PATH)

print("✓ Workflow function ready to use")

## 9. Summary

### What We've Learned

In this notebook, we've built a comprehensive music library management system with:

1. **Library Scanning**: Efficiently scan and catalog your entire music collection
2. **Smart Search**: Find songs before downloading to avoid duplicates
3. **Duplicate Detection**: Identify duplicates by file hash, size, or name similarity
4. **Auto-Organization**: Intelligently suggest and move files to appropriate folders
5. **Batch Renaming**: Clean up filenames with pattern matching and templates
6. **Batch Moving**: Organize files based on patterns and similarity

### Key Features

- **Dry-run mode**: Preview all operations before executing
- **Pattern matching**: Use regex for powerful file operations
- **Confidence scores**: See how confident the system is about suggestions
- **Safe operations**: Always checks if target exists before moving/renaming

### Best Practices

1. **Always use dry-run first**: Preview changes before applying them
2. **Backup your library**: Keep backups before bulk operations
3. **Start with small batches**: Test on a subset before processing everything
4. **Adjust thresholds**: Fine-tune similarity thresholds for your needs
5. **Regular maintenance**: Run duplicate detection periodically

### Next Steps

Enhance this notebook by:
- Adding metadata extraction (ID3 tags, album art)
- Implementing playlist management
- Creating visualizations of your music collection
- Adding audio analysis features
- Integrating with music streaming APIs

### Additional Resources

- [Python pathlib documentation](https://docs.python.org/3/library/pathlib.html)
- [Regular expressions guide](https://docs.python.org/3/howto/regex.html)
- [Mutagen - Audio metadata library](https://mutagen.readthedocs.io/)
- [MusicBrainz - Music metadata database](https://musicbrainz.org/)